In [1]:
import mlx.core as mx
import mlx.nn as nn

In [48]:
class MessagePassing(nn.Module):
    def __init__(self, aggr=None):
        self.aggr = aggr

    def __call__(self, x, edge_index, **kwargs):
        pass

    def propagate(self, x, edge_index, **kwargs):
        # process arguments and create *_kwargs
        
        src_idx, dst_idx = edge_index
        x_i = x[src_idx]
        x_j = x[dst_idx]

        # Message
        messages = self.message(x_i, x_j) #**msg_kwargs)

        # Aggregate
        aggregated = self.aggregate(messages, dst_idx) #**agg_kwargs)

        # Update
        output = self.update(aggregated) #**upd_kwargs)

        return output

    def message(self, x_i, x_j, **kwargs):
        return x_i

    def aggregate(self, messages, indices, **kwargs):
        if self.aggr == "add":
            nb_unique_indices = _unique(indices)
            empty_tensor = mx.zeros((nb_unique_indices, messages.shape[-1]))
            update_dim = (messages.shape[0], 1, messages.shape[1])
            return mx.scatter_add_(empty_tensor, [indices], messages.reshape(update_dim), [0], None)

    def update(self, aggregated, **kwargs):
        raise NotImplementedError

    
def _unique(array):
    return len(set(array.tolist()))

In [49]:
class GCN(MessagePassing):
    def __init__(self, x_dim, h_dim):
        super().__init__(aggr="add")
        
        self.linear = nn.Linear(x_dim, h_dim)

    def __call__(self, x, edge_index, **kwargs):
        # x = self.linear(x)
        x = self.propagate(x=x, edge_index=edge_index)

        return x

In [50]:
gcn = GCN(1, 32)

x = mx.array([[1], [2], [3], [4]])
edge_index = mx.array([
    [0, 0, 0, 1, 2],
    [1, 2, 3, 0, 0]
])
# expect: [5, 1, 1, 1]

gcn(x, edge_index)